In [ ]:
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from SimpleBrainMesh import generate_doughnut_mesh
from DarcySolver import solve_darcy_on_doughnut_brain

T = 5.0           # final time
num_steps = 100    # number of time steps
dt = T / num_steps

N = 30 # resolution
brain_radius = 0.1  # brain circle Radius
ventricle_radius = brain_radius/3
aqueduct_width = brain_radius/5

kappa = 5*1e-8    # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity

a_0 = 0.6
a_1 = 1.5
f_0 = 1.0/(60/15)
f_1 = 1.0/(60/70)
c = 7 
mmHg2Pa = 133.32


In [ ]:
# create mesh

mesh = Mesh(generate_doughnut_mesh(brain_radius, ventricle_radius, aqueduct_width, N))

probe_points = [
                Point(ventricle_radius + 0.2*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.4*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.6*(brain_radius - ventricle_radius), 0),
                Point(ventricle_radius + 0.8*(brain_radius - ventricle_radius), 0)
                ]

x_coords = np.linspace(ventricle_radius, brain_radius, 1000)
slice_points = [Point(x, 0.0) for x in x_coords]

p_obs = Expression("""c*mmHg2Pa*(a_0 * sin(2 *pi * f_0*t) + a_1 *sin(2*pi*f_1*t))
                   *( sqrt( x[0]*x[0] + x[1]*x[1] ) - R_ventricle ) """,
                    a_0=a_0, a_1=a_0, f_0=f_0, f_1=f_1, t = 0,c=c,
                    mmHg2Pa=mmHg2Pa, R_ventricle=ventricle_radius,
                    degree = 2)
p_skull = p_obs
p_ventricle = p_obs

f = Constant(0.0)
results = solve_darcy_on_doughnut_brain(mesh, f, T, num_steps, K=K,
                                        probe_points=probe_points,
                                        slice_points=slice_points,
                                        dirichlet_boundary_skull=p_skull,
                                        dirichlet_boundary_ventricle=p_ventricle)
                                            

In [ ]:
plt.figure(figsize=(10,10))
num_points = len(probe_points)
for i in range(num_points):
    plt.plot(np.linspace(0,T, num_steps), 
             results["probe_point_data"][:,i], 
             label="initial $P_{}$".format(i)
            )
plt.xlabel("t in s")
plt.ylabel("p in Pa")
plt.grid()
plt.legend()


In [ ]:
plt.figure(figsize=(12,12))
cont = plot(mesh)
for i, point in enumerate(probe_points):
    plt.scatter(point.array()[0], point.array()[1],
                marker=".", s= 200)
    plt.annotate( "$P_{}$".format(i), point.array()[0:2], size=20)

In [ ]:
times = [1,2,3,4,5]

idx = []
for i in range(num_steps):
    if np.isclose((i + 1)*dt, times).any():
        idx.append(i)
cross_section = results["slice"]
plt.figure(figsize=(10,10))
for i in idx:
    plt.plot(x_coords, cross_section[i,:], label="t = {}".format((i +1)*dt))
plt.legend()
plt.grid()
plt.xlabel("x in m")
plt.ylabel("p in Pa")